#### Install Dependencies

In [1]:
!pip install -q fastapi uvicorn langchain>=1.0.4 langgraph langchain-google-genai chromadb mlflow pydantic

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.80 which is incompatible.
streamlit 1.43.0 requires protobuf<6,>=3.20, but you have protobuf 6.33.1 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Configuration

In [2]:
import os

# ---- GCP / Gemini Config ----
PROJECT_ID = os.getenv("GCP_PROJECT_ID", "your-project-id")
REGION = "us-central1"
GEMINI_MODEL = "gemini-2.0-flash"

# ---- Local Paths (for testing) ----
SQLITE_DB_PATH = "LoanDB_BlueLoans4all.sqlite"     # place your sqlite db here
CHROMA_PATH = "chroma_store"   # pre-fed chroma directory

print("Config loaded")

Config loaded


#### ML Flow setup

In [3]:
import mlflow

mlflow.set_tracking_uri("http://20.75.92.162:5000/")
mlflow.set_experiment("Sid_test_loan_navigator")

<Experiment: artifact_location='mlflow-artifacts:/548246247213487410', creation_time=1765953380858, experiment_id='548246247213487410', last_update_time=1765953380858, lifecycle_stage='active', name='Sid_test_loan_navigator', tags={}>

In [5]:
from pydantic import BaseModel
from typing import Optional, Dict, Any

class LoanAgentState(BaseModel):
    user_query: str
    intent: Optional[str] = None

    sql_result: Optional[dict] = None
    policy_result: Optional[str] = None
    policy_confidence: Optional[float] = None
    calc_result: Optional[dict] = None

    errors: Optional[str] = None
    final_answer: Optional[str] = None


#### Intent Classification

In [6]:
def supervisor_intent_classifier(question: str) -> str:
    q = question.lower()

    if any(k in q for k in ["rbi", "policy", "rule", "guideline"]):
        return "POLICY_ONLY"

    if any(k in q for k in ["prepay", "what if", "close early"]):
        return "SQL_THEN_CALC"

    if any(k in q for k in ["eligible", "topup"]):
        return "SQL_THEN_POLICY"

    if any(k in q for k in ["emi", "amount", "due", "status"]):
        return "SQL_ONLY"

    return "CLARIFICATION"


In [7]:
def supervisor_router(state: LoanAgentState):
    intent = supervisor_intent_classifier(state.user_query)
    state.intent = intent

    mlflow.log_param("intent", intent)
    return intent


In [8]:
import sqlite3

def run_sql(query: str):
    conn = sqlite3.connect(SQLITE_DB_PATH)
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    conn.close()

    if not rows:
        return None
    return dict(rows[0])

In [9]:

from langchain.chat_models import init_chat_model

llm = init_chat_model(
    GEMINI_MODEL,
    model_provider="google_genai"
)

/home/zadmin/Desktop/B6_GAAP_GCP/agenticai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from langchain_core.prompts import ChatPromptTemplate


SQL_PROMPT = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a fintech SQL generator for a loan support system.\n"
        "CRITICAL RULES:\n"
        "1. Output ONLY a single SQL SELECT statement.\n"
        "2. Do NOT add explanations, comments, or markdown.\n"
        "3. Use ONLY this table: loan_data\n"
        "4. Allowed columns:\n"
        "   loan_id, customer_id, loan_amount, interest_rate,\n"
        "   start_date, tenure_months, monthly_emi, amount_paid,\n"
        "   next_due_date, status, topup_eligible, prepayment_limit\n"
        "5. NEVER invent table or column names.\n"
        "6. If the question cannot be answered, output exactly:\n"
        "   SELECT NULL;\n"
    ),
    ("human", "{question}")
])

In [11]:
import re

ALLOWED_TABLES = {"loan_data"}
ALLOWED_COLUMNS = {
    "loan_id", "customer_id", "loan_amount", "interest_rate",
    "start_date", "tenure_months", "monthly_emi", "amount_paid",
    "next_due_date", "status", "topup_eligible", "prepayment_limit"
}

def extract_sql(text: str) -> str:
    text = text.strip()
    text = re.sub(r"```sql|```", "", text, flags=re.IGNORECASE).strip()
    match = re.search(r"(SELECT\s.+)", text, re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else ""

def sql_agent(state: LoanAgentState):
    chain = SQL_PROMPT | llm
    raw_output = chain.invoke({"question": state.user_query}).content

    

    sql_query = extract_sql(raw_output)

    mlflow.log_text(sql_query, "generated_sql.sql")

    if not sql_query.lower().startswith("select"):
        mlflow.log_param("sql_error", "unsafe_sql")
        return {"errors": f"Unsafe SQL output: {raw_output}"}

    # Table guard
    if not any(t in sql_query.lower() for t in ALLOWED_TABLES):
        return {"errors": "Unauthorized table referenced"}

    # Block destructive ops
    if any(k in sql_query.lower() for k in ["insert", "update", "delete", "drop"]):
        return {"errors": "Write operations are not allowed"}

    result = run_sql(sql_query)
    return {"sql_result": result}

In [ ]:
def calculate_emi(P, r, n):
    r = r / (12 * 100)
    return (P * r * (1 + r)**n) / ((1 + r)**n - 1)

# def calculator_agent(state: LoanAgentState):
#     data = state.sql_result

#     if not data:
#         return {"errors": "Loan data not found"}

#     return {
#         "calc_result": {
#             "monthly_emi": data.get("monthly_emi"),
#             "prepayment_limit": data.get("prepayment_limit"),
#             "topup_eligible": bool(data.get("topup_eligible"))
#         }
#     }

# def calculator_agent(state: LoanAgentState):
#     data = state.sql_result
#     if not data:
#         mlflow.log_param("calc_error", "missing_sql_data")
#         return {"errors": "Loan data missing for calculation"}

#     result = {
#         "monthly_emi": data.get("monthly_emi"),
#         "prepayment_limit": data.get("prepayment_limit"),
#         "topup_eligible": bool(data.get("topup_eligible"))
#     }

#     mlflow.log_dict(result, "calculator_result.json")
#     return {"calc_result": result}

def extract_prepayment_amount(query: str):
    match = re.search(r"₹?\s*(\d+)", query)
    return float(match.group(1)) if match else None


def calculator_agent(state: LoanAgentState):
    data = state.sql_result
    prepay = extract_prepayment_amount(state.user_query)

    if not data or not prepay:
        return {"errors": "Missing loan data or prepayment amount"}

    outstanding = data["loan_amount"] - data["amount_paid"] - prepay
    new_emi = calculate_emi(
        outstanding,
        data["interest_rate"],
        data["tenure_months"]
    )

    result = {
        "old_emi": data["monthly_emi"],
        "new_emi": round(new_emi, 2),
        "outstanding_after_prepay": round(outstanding, 2)
    }

    mlflow.log_dict(result, "calculator_result.json")
    return {"calc_result": result}


In [16]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

def policy_agent(state: LoanAgentState):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectordb = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embeddings
    )

    docs = vectordb.similarity_search(state.user_query, k=3)
    if not docs:
        return {"errors": "No relevant policy found"}

    context = "\n".join([d.page_content for d in docs])

    PROMPT = ChatPromptTemplate.from_messages([
        ("system", "Answer strictly using policy context. Cite when possible."),
        ("human", "Question: {question}\nContext:\n{context}")
    ])

    chain = PROMPT | llm
    answer = chain.invoke({
        "question": state.user_query,
        "context": context
    }).content

    mlflow.log_text(answer, "policy_answer.txt")
    

    return {"policy_result": answer}

In [13]:
def after_sql_router(state: LoanAgentState):
    if state.errors:
        return "synthesize"

    if state.intent == "SQL_THEN_CALC":
        return "calculator"

    if state.intent == "SQL_THEN_POLICY":
        return "policy"

    return "synthesize"


In [ ]:
def response_synthesizer(state: LoanAgentState):
    if state.final_answer:
        return {"final_answer": state.final_answer}

    if state.errors:
        return {"final_answer": f"{state.errors}"}

    # if state.calc_result:
    #     return {"final_answer": f"Your EMI is ₹{state.calc_result['monthly_emi']}."}

    if state.calc_result:
        return {
            "final_answer": (
                f"Your current EMI is ₹{state.calc_result['old_emi']}. "
                f"After prepayment, your EMI will be approximately "
                f"₹{state.calc_result['new_emi']}."
            )
        }


    if state.sql_result:
        return {"final_answer": f"Here are your loan details: {state.sql_result}"}

    if state.policy_result:
        return {"final_answer": state.policy_result}

    return {"final_answer": "Could you please clarify your question?"}


In [ ]:
# from langgraph.graph import StateGraph, START

# graph = StateGraph(LoanAgentState)

# graph.add_node("sql", sql_agent)
# graph.add_node("calculator", calculator_agent)
# graph.add_node("policy", policy_agent)
# graph.add_node("synthesize", response_synthesizer)

# # ---- Supervisor Entry ----
# graph.add_conditional_edges(
#     START,
#     supervisor_router,
#     {
#         "sql": "sql",
#         "policy": "policy",
#         "sql_then_calculator": "sql",
#         "sql_then_policy": "sql",
#     }
# )

# # ---- After SQL ----
# graph.add_conditional_edges(
#     "sql",
#     after_sql_router,
#     {
#         "calculator": "calculator",
#         "policy": "policy",
#         "synthesize": "synthesize",
#     }
# )

# # ---- Terminal ----
# graph.add_edge("calculator", "synthesize")
# graph.add_edge("policy", "synthesize")

# loan_graph = graph.compile()

In [19]:
from langgraph.graph import StateGraph, START

# --------------------------------------------------
# Initialize Graph
# --------------------------------------------------
graph = StateGraph(LoanAgentState)

# --------------------------------------------------
# Register Nodes
# --------------------------------------------------
graph.add_node("sql", sql_agent)
graph.add_node("calculator", calculator_agent)
graph.add_node("policy", policy_agent)
graph.add_node("synthesize", response_synthesizer)

# --------------------------------------------------
# START → Supervisor Routing
# (MUST cover ALL possible supervisor outputs)
# --------------------------------------------------
graph.add_conditional_edges(
    START,
    supervisor_router,
    {
        # ---- SQL paths ----
        "SQL_ONLY": "sql",
        "SQL_THEN_CALC": "sql",
        "SQL_THEN_POLICY": "sql",

        # ---- Policy-only path (NO SQL) ----
        "POLICY_ONLY": "policy",

        # ---- Clarification / fallback ----
        "CLARIFICATION": "synthesize",
    }
)

# --------------------------------------------------
# After SQL → Decide next step
# --------------------------------------------------
graph.add_conditional_edges(
    "sql",
    after_sql_router,
    {
        "calculator": "calculator",
        "policy": "policy",
        "synthesize": "synthesize",
    }
)

# --------------------------------------------------
# Terminal Edges
# --------------------------------------------------
graph.add_edge("calculator", "synthesize")
graph.add_edge("policy", "synthesize")

# --------------------------------------------------
# Compile Graph
# --------------------------------------------------
loan_graph = graph.compile()


In [22]:
with mlflow.start_run(run_name="loan_navigator_query"):
    state = LoanAgentState(user_query="If loan id 2002 prepays ₹200, how will the EMI change?")
    result = loan_graph.invoke(state.dict())

result

/tmp/ipykernel_7478/861650697.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result = loan_graph.invoke(state.dict())


🏃 View run loan_navigator_query at: http://20.75.92.162:5000/#/experiments/548246247213487410/runs/118808ca11634c759a71a8123567e3f5
🧪 View experiment at: http://20.75.92.162:5000/#/experiments/548246247213487410


{'user_query': 'If loan id 2002 prepays ₹200, how will the EMI change?',
 'intent': None,
 'sql_result': None,
 'policy_result': None,
 'policy_confidence': None,
 'calc_result': None,
 'errors': 'Unauthorized table referenced',
 'final_answer': '⚠️ Unauthorized table referenced'}